In [11]:
import boto3
import os
import json
from pprint import pprint
from dotenv import load_dotenv

In [2]:
# 認証情報を環境変数から読み込み
load_dotenv()

True

In [3]:
# 認証情報を直接指定
session = boto3.Session(
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

### 2つのナレッジベースにアクセスする準備を実施

In [4]:
region_name = "us-east-1"
# ナレッジベースID chunk 100分割の方
knowledge_base_id1 = "EBUXJ8RLCT"
# ナレッジベースID デフォルト 300分割の方
knowledge_base_id2 = "YFB3JIZ2VQ"

kb_client_runtime = session.client("bedrock-agent-runtime", region_name=region_name)
bedrock_runtime = session.client(service_name="bedrock-runtime", region_name=region_name)

# 推論用モデル(今回はClaudeのv2を使います)
model_id = "anthropic.claude-v2"
# クエリ拡張用のモデル(Claude Instantを使います。これは簡単なタスクにおける速度向上が狙いです)
q_model_id = "anthropic.claude-instant-v1"

In [5]:
# 推論
# 推論用コード
def invoke_claude(text, model_id, max_tokens_to_sample=1000):
    body = json.dumps({
        "prompt": f"\n\nHuman:{text}\n\nAssistant: ",
        "max_tokens_to_sample": max_tokens_to_sample,
        "temperature": 0.1,
        "top_p": 0.9,
    })
    accept = "application/json"
    content_type = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())
    return response_body.get('completion')[1:]

# クエリ拡張用コード
def generate_queries(original_query, n=4):
    f = ''
    for i in range(n):
        f += f'{n}: \n'

    prompt = f'''
以下に示すQueryはユーザの入力した検索クエリです。
このクエリに関連するクエリを多角的な視点から{n}つ生成してください
Formatに従って結果を出力してください

<Query>
{original_query}
</Query>

<Format>
{f}
</Format>
'''
    result = invoke_claude(prompt, q_model_id)
    result = result.split('<Format>')[1].split('</Format>')[0]
    # print(result)
    generated_queries = []
    for q in result.split('\n'):
        print("拡張クエリ", q)
        if q == '':
            continue
        generated_queries.append(q.split(' ')[1])
    # ここでなぜか解答を真ん中2つに絞っている？　確かに解答前後は空白ではあるが？
    return generated_queries[1:-1]

In [10]:
# RAG検索用コード
def kb_search(query, knowledge_base_id, n=5):
    res = kb_client_runtime.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=knowledge_base_id,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': n
            }
        }
    )

    # {doc: score}の形に整形します
    return_dict = {}
    for r in res['retrievalResults']:
        return_dict[r['content']['text']] = r['score']

    return return_dict

In [7]:
# RRFのコード
def reciprocal_rank_fusion(all_results, k=50):
    fused_scores = {}
    for query, doc_scores in all_results.items():
        for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
            if doc not in fused_scores:
                fused_scores[doc] = 0
            previous_score = fused_scores[doc]
            fused_scores[doc] += 1 / (rank + k)

    reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
    return reranked_results

In [16]:
# 問い合わせprompt
original_guery = """
経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください
財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください
"""

queries = generate_queries(original_guery)
queries.insert(0, original_guery)
print(f"queryの一覧：{queries}")


拡張クエリ 
拡張クエリ 4: 財務分析方法 
拡張クエリ 4: 企業価値評価方法
拡張クエリ 4: 上場基準
拡張クエリ 4: 株式市場
拡張クエリ 
拡張クエリ 
queryの一覧：['\n経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください\n財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください\n', '企業価値評価方法', '上場基準']


In [17]:
# ragでのナレッジ検索結果を比較
all_result_chunk_100 = {}
all_result_chunk_300 = {}
for q in queries:
    all_result_chunk_100[q] = kb_search(q, knowledge_base_id1)

for q in queries:
    all_result_chunk_300[q] = kb_search(q, knowledge_base_id2)
    
pprint(all_result_chunk_100)
print("=========================================")
pprint(all_result_chunk_300)



{'\n経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について教えてください\n財政状態及び経営成績の状態について、アームがNasdaq Grobal Select Marketへ上場したと思うのですが、そちらについて詳しい説明をしてください\n': {'アリババ Alibaba Group Holding Limited   MgmtCo MASA USA LLC       （注）2023年８月にArm Limitedの子会社であったArm Holdings LimitedがArm Limitedの発行済普通株式の全   てを取得し、同社を完全子会社化する組織再編が行われました。その後、Arm Holdings Limitedは社名   をArm Holdings plcに変更し、2023年９月14日に新規株式公開でNasdaq Global Select Marketへ上場   しました。       EDINET提出書類   ソフトバンクグループ株式会社(E02778)   四半期報告書    61/112        ３．重要性がある会計方針   本要約四半期連結財務諸表において適用する重要性がある会計方針は、2023年３月31日に終了した１年間の連   結財務諸表において適用した会計方針と同一です。なお、2023年12月31日に終了した９カ月間における法人所得   税は、年間の見積実効税率に基づいて算定しています。': 0.56578684,
                                                                                                                                       '・期末日の対米ドルの為替換算レートが6.2％円安となったことにより外貨建普通社債の帳簿価額が   増加しました。       資金調達を行う100％子会社   Ｃ 借入金 2023年９月のアーム上場前に、アーム株式を利用したアセットバック・ファイナンスによる借入金   85.0億米ドル（前期末残高は１兆1,266億円）を返済し、上場後に再度アーム株式を利用したマージ   ンローンにより85.0億米ドル（当第３四半期末残高は１兆1,

In [19]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_result_chunk_100).keys())[:5]
print(reranked_results)
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'
"""
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。
"""

prompt = """
あなたはSBGの有価証券報告書を熟読した証券マンです
あなたはSBGの株を買おうか悩んでいる新規ユーザの疑問を聞いて、それについて回答する義務があります。
あなたのために、Informationセクションに、実際の有価証券報告書から取得できたナレッジを提供しますので、有効活用してください


<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_guery)
print(f"""入力前最終プロンプトはこちら
      {prompt}
      """)


['（注２）発行済株式は、全額払込済となっています。       （２）資本剰余金   2023年12月31日に終了した９カ月間   2023年９月14日のアームの新規株式公開において、当社は100％子会社を通じて保有するアーム株式の一部   （発行済株式総数の10.0％）を売り出しました。この結果、当社のアーム株式の保有割合は90.0％となりまし   た。   この取引に伴い、連結上のアーム株式売却益相当額674,370百万円を「支配継続子会社に対する持分変動」   として「資本剰余金」に計上しています。       （３）その他の資本性金融商品   ソフトバンクグループ㈱は2017年７月19日に、米ドル建ノンコール６年永久劣後特約付社債（利払繰延条項   付）27.5億米ドルおよび米ドル建ノンコール10年永久劣後特約付社債（利払繰延条項付）17.5億米ドル（以下   あわせて「本ハイブリッド社債」）を発行しました。   本ハイブリッド社債は、利息の任意繰延が可能であり償還期限の定めがなく、清算による残余財産の分配時   を除き現金またはその他の資本性金融資産の引渡しを回避する無条件の権利を有していることから、IFRS上資   本性金融商品に分類されます。       ソフトバンクグループ㈱は上記米ドル建ノンコール６年永久劣後特約付社債について、2022年10月12日に一   部（額面7.5億米ドル）について買入れを行い同日に消却し、初回任意償還日である2023年７月19日に残りの   全額（額面20億米ドル）を償還しました。', '科目 前期末からの主な増減理由   Ａ 資本剰余金 当第２四半期に、アームの上場に伴う売出しにより売却益相当額674,370百万円（46.5   億米ドル）を計上しました。本取引の詳細は「（１）財政状態及び経営成績の状況＜   アーム株式のグループ内取引およびアームの新規株式公開＞」をご参照ください。       Ｂ その他の資本性金融商品 当第２四半期に、初回任意償還日を迎えた米ドル建ノンコール６年永久ハイブリッド社   債（20.0億米ドル）を償還しました。本社債は、IFRS上資本性金融商品に分類されてい   ました。       Ｃ 利益剰余金 親会社の所有者に帰属する純損失458,723百万円を計上しました。       Ｄ 

In [20]:
print(invoke_claude(prompt, model_id))

はい、アームのNasdaq上場に関する財政状態及び経営成績の状況について説明します。

アームはソフトバンクグループの子会社であるArm Holdings plcのことです。アームは2022年9月にNasdaqグローバルセレクト市場に上場しました。 

上場に先立ち、ソフトバンクグループはアーム株式の一部(発行済株式総数の10.0%)を売却しました。この結果、ソフトバンクグループのアーム株式の保有割合は90.0%となりました。

このアーム株式の売却により、連結上674,370百万円の売却益が発生し、これを「支配継続子会社に対する持分変動」として資本剰余金に計上しています。

また、アームの上場に伴い、アーム株式を担保とする既存の借入金を全額返済し、上場後にアーム株式を担保とする新たなマージンローンにより85億米ドルを借り入れています。

このように、アームの上場により資本剰余金が増加する一方、アーム株式を利用した資金調達の方法が変更されるなど、ソフトバンクグループの財務状況に一定の影響が生じています。


In [21]:
# ここでは上位5件を採用している
reranked_results = list(reciprocal_rank_fusion(all_result_chunk_300).keys())[:5]
print(reranked_results)
information = ''
for i, r in enumerate(reranked_results):
    information += f'情報{i+1}. {r}\n'
"""
日本では本日(2月16日)から確定申告の相談及び申告書の受付がスタートしましたね。
この国の風物詩であるこの確定申告という儀式では、毎年数多の迷える子羊が出現することで有名です。
あなたは全知全能の税神として、迷える子羊の抱いている確定申告に対する悩みを聞き、彼らを正しき納税へと導いてください。
全知全能たるあなたのために、Informationセクションには迷える子羊の悩みに関連しそうな情報を提供します。有効に活用してください。
"""

prompt = """
あなたはSBGの有価証券報告書を熟読した証券マンです
あなたはSBGの株を買おうか悩んでいる新規ユーザの疑問を聞いて、それについて回答する義務があります。
あなたのために、Informationセクションに、実際の有価証券報告書から取得できたナレッジを提供しますので、有効活用してください


<Information>
{information}
</Information>

<UserQuery>
{query}
</UserQuery>
"""
prompt = prompt.format(information=information, query=original_guery)
print(f"""入力前最終プロンプトはこちら
      {prompt}
      """)


['なお、2023年６月１日に当該権利が確定し、当社はTモバイル株式3,566,400株を受領しました。       （注４）2020年４月１日のスプリントとT-Mobile US, Inc.の合併取引により取得したものであり、一定の条   件を満たした際に当社がTモバイル株式48,751,557株を無償で取得する権利です。2023年12月22日に   当該条件を満たしたことに伴い、当社は2023年12月28日にTモバイル株式48,751,557株を無償で取得   しました。2023年12月31日現在、本取引により取得したTモバイル株式の公正価値は1,108,591百万円   であり、要約四半期連結財政状態計算書上、従来から保有するTモバイル株式とあわせて2,094,085百   万円を「投資有価証券」に含めて計上しています。   なお、上記の合併取引に関連して当社が引き受けた補償義務の履行に備えて、当社は本取引により取   得したTモバイル株式のうち18,000,000株（2023年12月31日現在の帳簿価額409,313百万円）を2025年   ４月１日まで担保に供しています。           EDINET提出書類   ソフトバンクグループ株式会社(E02778)   四半期報告書    83/112        経常的に公正価値で測定する金融商品の公正価値の主な測定方法は、以下の通りです。       a．SVFからの投資（FVTPL）、株式、債券および貸付金   活発な市場における同一銘柄の相場価格が入手できる場合の公正価値は、当該相場価格を使用して測定   し、レベル１に分類しています。   活発な市場における同一銘柄の相場価格が入手できない場合、直近の独立した第三者間取引やファイナ   ンス価格の情報が利用可能な場合は、公正価値はそのような直近の取引価格に基づき評価され、評価対象   銘柄の発行企業が属する市場動向や企業の業績によって調整されます。   これらの直近の取引情報が利用できない場合の企業価値評価には、マーケット・アプローチ、インカ   ム・アプローチ、またはネットアセット・アプローチを用いています。   マーケット・アプローチは、評価対象会社と比較可能な類似会社の情報が利用可能な場合に利用され、   評価対象会社の財務諸表数値と比較対象

In [22]:
print(invoke_claude(prompt, model_id))

経営者による財政状態、経営成績及びキャッシュ・フローの状況分析について、要約四半期報告書の「第3 経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析」に以下のような記載があります。

- 売上高は前年同期比で増加しており、ソフトバンク事業の売上高が増加したことが主な要因です。また、営業利益は前年同期を上回っています。 

- 有利子負債残高は前年同期比で増加しています。これは主に、アームの株式公開に伴う借入金の返済などによる影響です。

- キャッシュ・フローについては、営業活動によるキャッシュ・フローは黒字で推移しています。一方、投資活動によるキャッシュ・フローはSVFへの投資などによりマイナスとなっています。

アームがNasdaqグローバルセレクト市場に上場した点については、以下のように記載があります。

- アームは2022年10月にNasdaqグローバルセレクト市場に上場しました。アームの株式公開により、当社は1兆円超の売却益を計上しました。

- アームの株式公開に伴い、当社はアーム株式の一部を売却しましたが、引き続きアームの大株主として支配力を保持しています。

- アームの株式公開により調達した資金の一部は、有利子負債の返済に充当しました。

以上のように、アームの株式公開に伴う影響として、売却益の計上や有利子負債の返済などが記載されています。
